In [ ]:


!pip install qiskit qiskit-aer matplotlib pylatexenc ipywidgets


In [ ]:

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_bloch_multivector
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display


In [ ]:

def build_superdense_circuit(msg: str) -> QuantumCircuit:
    if msg not in {"00", "01", "10", "11"}:
        raise ValueError("msg must be one of: '00', '01', '10', '11'")

    qc = QuantumCircuit(2, 2, name=f"SDC_{msg}")

    # Step 1: Entanglement (Bell pair)
    qc.h(0)
    qc.cx(0, 1)

    # Step 2: Alice encodes her 2-bit message
    if msg == "01":
        qc.x(0)
    elif msg == "10":
        qc.z(0)
    elif msg == "11":
        qc.x(0)
        qc.z(0)
    # "00" -> do nothing

    # Step 3: Bob decodes
    qc.cx(0, 1)
    qc.h(0)

    # Step 4: Measure
    qc.measure([0, 1], [0, 1])
    return qc


In [ ]:

def run_superdense_simulation(msg: str, shots: int = 1024, seed=None):
    qc = build_superdense_circuit(msg)
    sim = AerSimulator(seed_simulator=seed)
    tqc = transpile(qc, sim, optimization_level=1, seed_transpiler=seed)
    result = sim.run(tqc, shots=shots).result()
    counts = result.get_counts()
    return qc, counts


## Trying One Message

In [ ]:

msg = "10"  # Change to "00", "01", "10", or "11"
qc, counts = run_superdense_simulation(msg)
display(qc.draw("mpl"))
plot_histogram(counts, title=f"Decoded result for message {msg}")
plt.show()
print("Counts:", counts)


## Running All Messages

In [ ]:

def demo_all_messages(shots=1024, seed=None):
    results = {}
    for msg in ["00", "01", "10", "11"]:
        qc, counts = run_superdense_simulation(msg, shots, seed)
        display(qc.draw("mpl"))
        plot_histogram(counts, title=f"Decoded result for message {msg}")
        plt.show()
        print(f"Message {msg} -> Counts: {counts}")
        results[msg] = counts
    return results

demo_all_messages()
